In [1]:
from pathlib import Path

import gym_pusht  # noqa: F401
import gymnasium as gym
import imageio
import numpy as np
import torch
from huggingface_hub import snapshot_download

from lerobot.common.policies.diffusion.modeling_diffusion import DiffusionPolicy



/home/lawrence/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#  Create a directory to store the video of the evaluation
output_directory = Path("outputs/eval/example_pusht_diffusion")
output_directory.mkdir(parents=True, exist_ok=True)

# Download the diffusion policy for pusht environment
pretrained_policy_path = Path(snapshot_download("lerobot/diffusion_pusht"))
# OR uncomment the following to evaluate a policy from the local outputs/train folder.
# pretrained_policy_path = Path("outputs/train/2024-11-26/18-20-16_pusht_diffusion_default")

policy = DiffusionPolicy.from_pretrained(pretrained_policy_path)
policy.eval()

# Check if GPU is available
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available. Device set to:", device)
else:
    device = torch.device("cpu")
    print(f"GPU is not available. Device set to: {device}. Inference will be slower than on GPU.")
    # Decrease the number of reverse-diffusion steps (trades off a bit of quality for 10x speed)
    policy.diffusion.num_inference_steps = 10

policy.to(device)



Fetching 11 files: 100%|██████████| 11/11 [00:00<00:00, 73002.13it/s]


Loading weights from local directory
GPU is available. Device set to: cuda


DiffusionPolicy(
  (normalize_inputs): Normalize(
    (buffer_observation_image): ParameterDict(
        (mean): Parameter containing: [torch.cuda.FloatTensor of size 3x1x1 (cuda:0)]
        (std): Parameter containing: [torch.cuda.FloatTensor of size 3x1x1 (cuda:0)]
    )
    (buffer_observation_state): ParameterDict(
        (max): Parameter containing: [torch.cuda.FloatTensor of size 2 (cuda:0)]
        (min): Parameter containing: [torch.cuda.FloatTensor of size 2 (cuda:0)]
    )
  )
  (normalize_targets): Normalize(
    (buffer_action): ParameterDict(
        (max): Parameter containing: [torch.cuda.FloatTensor of size 2 (cuda:0)]
        (min): Parameter containing: [torch.cuda.FloatTensor of size 2 (cuda:0)]
    )
  )
  (unnormalize_outputs): Unnormalize(
    (buffer_action): ParameterDict(
        (max): Parameter containing: [torch.cuda.FloatTensor of size 2 (cuda:0)]
        (min): Parameter containing: [torch.cuda.FloatTensor of size 2 (cuda:0)]
    )
  )
  (diffusion): Diff

In [6]:
# Initialize evaluation environment to render two observation types:
# an image of the scene and state/position of the agent. The environment
# also automatically stops running after 300 interactions/steps.
BATCH_SIZE = 100
env = gym.vector.make(
    "gym_pusht/PushT-v0",
    num_envs=BATCH_SIZE,
    obs_type="pixels_agent_pos",
    max_episode_steps= 200,
)

/home/lawrence/anaconda3/envs/lerobot/lib/python3.10/site-packages/gymnasium/vector/__init__.py:53: UserWarning: WARN: `gymnasium.vector.make(...)` is deprecated and will be replaced by `gymnasium.make_vec(...)` in v1.0
  gym.logger.warn(


In [ ]:
# Reset the policy and environments
BATCH_SIZE = 100
env = gym.vector.make(
    "gym_pusht/PushT-v0",
    num_envs=BATCH_SIZE,
    obs_type="pixels_agent_pos",
    max_episode_steps= 200,
)
policy.reset()
batch_observations, info = env.reset(seed=42)

# Prepare to collect rewards and frames for each environment in the batch
rewards = [[] for _ in range(BATCH_SIZE)]
frames = [[] for _ in range(BATCH_SIZE)]

# Render the initial frames
initial_frames = env.call("render")
for i in range(BATCH_SIZE):
    frames[i].append(initial_frames[i])

done = [False] * BATCH_SIZE
steps = [0] * BATCH_SIZE

while not all(done):
    # Prepare batched observations
    states = torch.from_numpy(batch_observations["agent_pos"]).to(torch.float32).to(device)
    images = torch.from_numpy(batch_observations["pixels"]).to(torch.float32) / 255
    images = images.permute(0, 3, 1, 2).to(device)  # Convert to channel-first format

    # Create batched input dictionary for the policy
    observations = {
        "observation.state": states,
        "observation.image": images,
    }

    # Predict actions in batch
    with torch.inference_mode():
        actions = policy.select_action(observations)

    numpy_actions = actions.cpu().numpy()

    # Step through all environments in the batch
    batch_observations, batch_rewards, batch_terminated, batch_truncated, infos = env.step(numpy_actions)

    # Track rewards, frames, and done status for each environment
    frames_ = env.call("render")
    for i in range(BATCH_SIZE):
        if not done[i]:
            rewards[i].append(batch_rewards[i])
            frames[i].append(frames_[i])
            done[i] = batch_terminated[i] or batch_truncated[i]
            steps[i] += 1
            print(f"Environment {i} has completed {steps[i]} steps. reward={batch_rewards[i]}")

# Encode videos for each environment
for i in range(BATCH_SIZE):
    video_path = output_directory / f"rollout_env_{i}.mp4"
    imageio.mimsave(str(video_path), np.stack(frames[i]), fps=env.metadata["render_fps"])
    print(f"Video for environment {i} is available at '{video_path}'.")

print("Batch evaluation complete.")

Environment 0 has completed 1 steps. reward=0.0
Environment 1 has completed 1 steps. reward=0.0
Environment 2 has completed 1 steps. reward=0.03055885971007947
Environment 3 has completed 1 steps. reward=0.017677395962375143
Environment 4 has completed 1 steps. reward=0.0002236514599414307
Environment 5 has completed 1 steps. reward=0.0
Environment 6 has completed 1 steps. reward=0.0
Environment 7 has completed 1 steps. reward=0.010031352981896939
Environment 8 has completed 1 steps. reward=0.0
Environment 9 has completed 1 steps. reward=0.0
Environment 10 has completed 1 steps. reward=0.0
Environment 11 has completed 1 steps. reward=0.0
Environment 12 has completed 1 steps. reward=0.0
Environment 13 has completed 1 steps. reward=0.0
Environment 14 has completed 1 steps. reward=0.0
Environment 15 has completed 1 steps. reward=0.24366805933367677
Environment 16 has completed 1 steps. reward=0.0
Environment 17 has completed 1 steps. reward=0.0904629963138023
Environment 18 has completed 

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (680, 680) to (688, 688) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
[swscaler @ 0x58fb000] Warning: data is not aligned! This can lead to a speed loss


Video for environment 0 is available at 'outputs/eval/example_pusht_diffusion/rollout_env_0.mp4'.


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (680, 680) to (688, 688) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
[swscaler @ 0x69d6000] Warning: data is not aligned! This can lead to a speed loss


Video for environment 1 is available at 'outputs/eval/example_pusht_diffusion/rollout_env_1.mp4'.


...

Video for environment 97 is available at 'outputs/eval/example_pusht_diffusion/rollout_env_97.mp4'.


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (680, 680) to (688, 688) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
[swscaler @ 0x61f5000] Warning: data is not aligned! This can lead to a speed loss


Video for environment 98 is available at 'outputs/eval/example_pusht_diffusion/rollout_env_98.mp4'.


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (680, 680) to (688, 688) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
[swscaler @ 0x5fba000] Warning: data is not aligned! This can lead to a speed loss


Video for environment 99 is available at 'outputs/eval/example_pusht_diffusion/rollout_env_99.mp4'.
Batch evaluation complete.


In [18]:
# Reset the policy and environmens to prepare for rollout

numpy_observation, info = env.reset(seed=1234)

# Prepare to collect every rewards and all the frames of the episode,
# from initial state to final state.
rewards = []
frames = []

# Render frame of the initial state
frames.append(env.render())

step = 0
done = False
while not done:
    # Prepare observation for the policy running in Pytorch
    state = torch.from_numpy(numpy_observation["agent_pos"])
    image = torch.from_numpy(numpy_observation["pixels"])

    # Convert to float32 with image from channel first in [0,255]
    # to channel last in [0,1]
    state = state.to(torch.float32)
    image = image.to(torch.float32) / 255
    image = image.permute(2, 0, 1)

    # Send data tensors from CPU to GPU
    state = state.to(device, non_blocking=True)
    image = image.to(device, non_blocking=True)

    # Add extra (empty) batch dimension, required to forward the policy
    state = state.unsqueeze(0)
    image = image.unsqueeze(0)

    # Create the policy input dictionary
    observation = {
        "observation.state": state,
        "observation.image": image,
    }
    
    # Predict the next action with respect to the current observation
    with torch.inference_mode():
        action = policy.select_action(observation)

    # Prepare the action for the environment
    numpy_action = action.squeeze(0).to("cpu").numpy()
    # print(numpy_action)
    # Step through the environment and receive a new observation
    numpy_observation, reward, terminated, truncated, info = env.step(numpy_action)
    print(f"{step=} {reward=} {terminated=}")

    # Keep track of all the rewards and frames
    rewards.append(reward)
    frames.append(env.render())

    # The rollout is considered done when the success state is reach (i.e. terminated is True),
    # or the maximum number of iterations is reached (i.e. truncated is True)
    done = terminated | truncated | done
    step += 1



step=0 reward=0.3549526479034709 terminated=False
step=1 reward=0.3549526479034709 terminated=False
step=2 reward=0.3549526479034709 terminated=False
step=3 reward=0.3549526479034709 terminated=False
step=4 reward=0.3549526479034709 terminated=False
step=5 reward=0.3549526479034709 terminated=False
step=6 reward=0.3549526479034709 terminated=False
step=7 reward=0.3549526479034709 terminated=False
step=8 reward=0.3549526479034709 terminated=False
step=9 reward=0.3549526479034709 terminated=False
step=10 reward=0.3549526479034709 terminated=False
step=11 reward=0.3549526479034709 terminated=False
step=12 reward=0.3549526479034709 terminated=False
step=13 reward=0.3583105184393292 terminated=False
step=14 reward=0.37652863787608654 terminated=False
step=15 reward=0.39283125939387686 terminated=False
step=16 reward=0.36670874447322566 terminated=False
step=17 reward=0.3135768316292473 terminated=False
step=18 reward=0.26680185903854986 terminated=False
step=19 reward=0.22731800821590223 te

In [19]:
if terminated:
    print("Success!")
else:
    print("Failure!")

# Get the speed of environment (i.e. its number of frames per second).
fps = env.metadata["render_fps"]

# Encode all frames into a mp4 video.
video_path = output_directory / "rollout.mp4"
imageio.mimsave(str(video_path), numpy.stack(frames), fps=fps)

print(f"Video of the evaluation is available in '{video_path}'.")

Failure!


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (680, 680) to (688, 688) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
[swscaler @ 0x58e8140] Warning: data is not aligned! This can lead to a speed loss


Video of the evaluation is available in 'outputs/eval/example_pusht_diffusion/rollout.mp4'.
